# Инструкция по загрузке данных и удобной работе с ними

Для начала импортируем файл с классом для загрузки данных.

Вместе с ним импортируются все необходимые для работы библиотеки.

In [1]:
%%time
import sys
sys.path.insert(0, 'basic_codes')

import class_magnesium_not_drop_na #_transform_pairings
from class_magnesium_not_drop_na import *

Wall time: 22.6 s


### Прописываем пути до файлов

In [2]:
fold = 'input_files/'
file_train = 'train.csv'
file_test = 'test.csv'

### Инициируем класс 

In [3]:
from catboost import CatBoostClassifier

In [4]:
from sklearn.ensemble import AdaBoostClassifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
%%time
'''
    Класс Magnesium загружает данные из файла file_, находящегося в папке fold. Делает dropna (чтобы кас не ругался)
    model - модель, с которой вы хотите работатьб по умолчанию RFC
    Сохраняет переменные:
    data - загруженный DataFrame
    features - признаки
    groups - цепочки
    x - данные для бучения в виде Numpy-матрицы
    y - целевой признак
'''
#mdl = CatBoostClassifier(iterations=500)#, learning_rate=1, depth=5, loss_function='Logloss', custom_metric='F1')
mdl = AdaBoostClassifier(n_estimators=666)
#mdl = RandomForestClassifier(n_estimators=1000)
m = Magnesium(file_train, fold = fold, model = mdl)   # Можно написать любую свою модель

Everything is OK. Ready for your experiments!Wall time: 1min 18s


In [5]:
pd.options.display.max_columns = 40

In [6]:
m.data.head()

,pdb_chain,xray,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,zetam2,e-zm2,chim2,phase-anglem2,sszpm2,dpm2,splaym2,etam2,thetam2,etapm2,...,atomO,atomN,atomN1,atomN2,atomN3,atomN4,atomN6,atomN7,atomN9,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
35,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,-71.0,-169.6,18.1,4.47,4.5,20.28,174.5,-148.6,-171.6,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
36,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,-71.0,-169.6,18.1,4.47,4.5,20.28,174.5,-148.6,-171.6,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
37,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,-71.0,-169.6,18.1,4.47,4.5,20.28,174.5,-148.6,-171.6,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
38,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,-71.0,-169.6,18.1,4.47,4.5,20.28,174.5,-148.6,-171.6,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
39,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,-71.0,-169.6,18.1,4.47,4.5,20.28,174.5,-148.6,-171.6,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [8]:
def load_data(filename):
    '''
        Функция для загрузки датасета без инициализации класса. 
        
        Возвращает:
         data - полный DataFrame
         fetures - список признаков для обучения/предсказания
         x - датасет для тренировки/предсказания в формате np.array
         y - целевой признак, если он есть. В случае с тестовым датасетом возвращает None
    '''
    
    data = pd.read_table(filename, sep=',').dropna()    
    
    data = data[~(data['chainlen']>1000)]

    if ('DSSR' in data.columns):
        data.drop('DSSR', axis=1, inplace=True)    
    
    features = list(deepcopy(data.columns))
    [features.remove(column) for column in ['Id','index', 'pdb_chain', 'mg'] if column in data.columns];
    x_test = np.array(data[features])
    
    try:
        y_test = np.array(data['mg'])
    except: 
        y_test = None
    change_output('Data loaded')
    return {'data':data, 'features':features, 'x':x_test, 'y':y_test}

### Тренируем модель на тренировочном датасете

In [6]:
%%time
m.trained_model = deepcopy(m.model)
m.trained_model.fit(m.x, m.y)

Wall time: 1h 22min 45s


In [9]:
test = load_data(fold+file_test)
pred = m.trained_model.predict(test['x'])
df = pd.DataFrame(pred, columns =['mg'])
df.index.name='Id'
df.to_csv('submission.csv')

Data loaded

In [13]:
df

,mg
Id,
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
